In [18]:
import pandas as pd
import glob
import os

# Get all csv files in the folder
folder_path = 'Aligned Data All Participants'

# Initialize separate dictionaries
axial_rotation = {}
flexion = {}
lateral_bending = {}

# Get all CSV files
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file in all_files:
    try:
        # Get the filename without path or extension
        filename = os.path.splitext(os.path.basename(file))[0]
        
        # Read the CSV file
        df = pd.read_csv(file)
        
        # Categorize based on filename
        if 'AxialRotation' in filename:
            axial_rotation[filename] = df
        elif 'Flexion' in filename:
            flexion[filename] = df
        elif 'LateralBending' in filename:
            lateral_bending[filename] = df
        else:
            print(f"File {filename} didn't match any category")
            
    except Exception as e:
        print(f"Error loading {file}: {str(e)}")

NameError: name 'folder_path' is not defined

In [16]:
dataframes.keys()

dict_keys(['10AxialRotation_IMU_95s_to_127s', '10AxialRotation_MoCap_95s_to_127s', '10Flexion_IMU_3s_to_44s', '10Flexion_MoCap_3s_to_44s', '10LateralBending_IMU_53s_to_85s', '10LateralBending_MoCap_53s_to_85s', '11AxialRotation_IMU_95s_to_127s', '11AxialRotation_MoCap_95s_to_127s', '11Flexion_IMU_3s_to_44s', '11Flexion_MoCap_3s_to_44s', '11LateralBending_IMU_53s_to_85s', '11LateralBending_MoCap_53s_to_85s', '12AxialRotation_IMU_95s_to_126s', '12AxialRotation_MoCap_95s_to_126s', '12Flexion_IMU_3s_to_44s', '12Flexion_MoCap_3s_to_44s', '12LateralBending_IMU_53s_to_85s', '12LateralBending_MoCap_53s_to_85s', '13AxialRotation_IMU_95s_to_126s', '13AxialRotation_MoCap_95s_to_126s', '13Flexion_IMU_3s_to_44s', '13Flexion_MoCap_3s_to_44s', '13LateralBending_IMU_53s_to_85s', '13LateralBending_MoCap_53s_to_85s', '14AxialRotation_IMU_95s_to_126s', '14AxialRotation_MoCap_95s_to_126s', '14Flexion_IMU_3s_to_44s', '14Flexion_MoCap_3s_to_44s', '14LateralBending_IMU_53s_to_85s', '14LateralBending_MoCap_53